# Step 1: Read the Parquet file into a DataFrame
chunks_df = spark.read.parquet("/Volumes/icc/jugement/files/chunks_data.parquet")

# Step 2: Write the DataFrame as a managed Delta table
chunks_df.write.format("delta").mode("overwrite").saveAsTable("icc.jugement.chunks")

# Step 3: Display the first few rows to confirm
chunks_table = spark.table("chunks")
display(chunks_table)

In [ ]:
spark.sql("ALTER TABLE icc.jugement.chunks SET TBLPROPERTIES (delta.enableChangeDataFeed = true)")

In [ ]:
from pyspark.sql.functions import expr

# Read the table as a DataFrame
chunks_df = spark.table("icc.jugement.chunks")

# Add the summary column using ai_summarize (limit to 200 tokens if needed)
chunks_with_summary = chunks_df.withColumn(
    "summary", expr("ai_summarize(content, 200)")
)

# Overwrite the table with the new DataFrame
chunks_with_summary.write.format("delta").mode("overwrite").option("overwriteSchema", "true").saveAsTable("icc.jugement.chunks")

# Display a sample of the updated table
display(spark.table("icc.jugement.chunks").select("chunk_id", "content", "summary").limit(10))